In [475]:
from bs4 import BeautifulSoup
from imdbpie import Imdb
import requests
import re
import pandas as pd
import nltk
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score, StratifiedKFold
import numpy as np

imdb = Imdb()
imdb = Imdb(anonymize=True) # to proxy requests

In [476]:
data = imdb.top_250()

In [477]:
data[0].keys()

[u'rating',
 u'tconst',
 u'title',
 u'image',
 u'num_votes',
 u'year',
 u'can_rate',
 u'type']

In [393]:
top_250_id_list = []
for i in data:
    top_250_id_list.append(i['tconst'])
#top_250_id_list

In [478]:
df = pd.DataFrame(data, columns=data[0].keys())

In [479]:
df

,rating,tconst,title,image,num_votes,year,can_rate,type
0,9.3,tt0111161,The Shawshank Redemption,{u'url': u'https://images-na.ssl-images-amazon...,1756854,1994,True,feature
1,9.2,tt0068646,The Godfather,{u'url': u'https://images-na.ssl-images-amazon...,1199825,1972,True,feature
2,9.0,tt0071562,The Godfather: Part II,{u'url': u'https://images-na.ssl-images-amazon...,823982,1974,True,feature
3,9.0,tt0468569,The Dark Knight,{u'url': u'https://images-na.ssl-images-amazon...,1739916,2008,True,feature
4,8.9,tt0050083,12 Angry Men,{u'url': u'https://images-na.ssl-images-amazon...,471357,1957,True,feature
5,8.9,tt0108052,Schindler's List,{u'url': u'https://images-na.ssl-images-amazon...,900235,1993,True,feature
6,8.9,tt0110912,Pulp Fiction,{u'url': u'https://images-na.ssl-images-amazon...,1375727,1994,True,feature
7,8.9,tt0167260,The Lord of the Rings: The Return of the King,{u'url': u'https://images-na.ssl-images-amazon...,1261755,2003,True,feature
8,8.9,tt0060196,"The Good, the Bad and the Ugly",{u'url': u'https://images-na.ssl-images-amazon...,522499,1966,True,feature
9,8.8,tt0137523,Fight Club,{u'url': u'https://images-na.ssl-images-amazon...,1403828,1999,True,feature


In [396]:
# Gets Box Office Info for each movie:
#df = df[0:10]
df['Box Office'] = df['tconst'].apply(get_box_office_info)
df.head()

,rating,tconst,title,image,num_votes,year,can_rate,type,Box Office
0,9.3,tt0111161,The Shawshank Redemption,{u'url': u'https://images-na.ssl-images-amazon...,1756854,1994,True,feature,"{u'Gross:': 28341469, u'Opening Weekend:': Non..."
1,9.2,tt0068646,The Godfather,{u'url': u'https://images-na.ssl-images-amazon...,1199825,1972,True,feature,"{u'Gross:': 134821952, u'Opening Weekend:': 22..."
2,9.0,tt0071562,The Godfather: Part II,{u'url': u'https://images-na.ssl-images-amazon...,823982,1974,True,feature,"{u'Gross:': 57300000, u'Opening Weekend:': Non..."
3,9.0,tt0468569,The Dark Knight,{u'url': u'https://images-na.ssl-images-amazon...,1739916,2008,True,feature,"{u'Gross:': 533316061, u'Opening Weekend:': 15..."
4,8.9,tt0050083,12 Angry Men,{u'url': u'https://images-na.ssl-images-amazon...,471357,1957,True,feature,"{u'Gross:': None, u'Opening Weekend:': None, u..."


In [406]:
#Sets each movies' genres as a list in a column:
df['Genres'] = df['tconst'].apply(get_genres)

In [407]:
df

,rating,tconst,title,image,num_votes,year,can_rate,type,Box Office,Budget,Gross,Opening Weekend,Genres
0,9.3,tt0111161,The Shawshank Redemption,{u'url': u'https://images-na.ssl-images-amazon...,1756854,1994,True,feature,"{u'Gross:': 28341469, u'Opening Weekend:': Non...",25000000,28341469.0,None,"[Crime, Drama]"
1,9.2,tt0068646,The Godfather,{u'url': u'https://images-na.ssl-images-amazon...,1199825,1972,True,feature,"{u'Gross:': 134821952, u'Opening Weekend:': 22...",6000000,134821952.0,223758,"[Crime, Drama]"
2,9.0,tt0071562,The Godfather: Part II,{u'url': u'https://images-na.ssl-images-amazon...,823982,1974,True,feature,"{u'Gross:': 57300000, u'Opening Weekend:': Non...",13000000,57300000.0,None,"[Crime, Drama]"
3,9.0,tt0468569,The Dark Knight,{u'url': u'https://images-na.ssl-images-amazon...,1739916,2008,True,feature,"{u'Gross:': 533316061, u'Opening Weekend:': 15...",185000000,533316061.0,158411483,"[Action, Crime, Drama, Thriller]"
4,8.9,tt0050083,12 Angry Men,{u'url': u'https://images-na.ssl-images-amazon...,471357,1957,True,feature,"{u'Gross:': None, u'Opening Weekend:': None, u...",350000,NaN,None,"[Crime, Drama]"
5,8.9,tt0108052,Schindler's List,{u'url': u'https://images-na.ssl-images-amazon...,900235,1993,True,feature,"{u'Gross:': 96067179, u'Opening Weekend:': u'£...",22000000,96067179.0,£1234591,"[Biography, Drama, History]"
6,8.9,tt0110912,Pulp Fiction,{u'url': u'https://images-na.ssl-images-amazon...,1375727,1994,True,feature,"{u'Gross:': 107930000, u'Opening Weekend:': u'...",8000000,107930000.0,£4243233,"[Crime, Drama]"
7,8.9,tt0167260,The Lord of the Rings: The Return of the King,{u'url': u'https://images-na.ssl-images-amazon...,1261755,2003,True,feature,"{u'Gross:': 377019252, u'Opening Weekend:': u'...",94000000,377019252.0,£15021761,"[Adventure, Drama, Fantasy]"
8,8.9,tt0060196,"The Good, the Bad and the Ugly",{u'url': u'https://images-na.ssl-images-amazon...,522499,1966,True,feature,"{u'Gross:': 6100000, u'Opening Weekend:': None...",1200000,6100000.0,None,[Western]
9,8.8,tt0137523,Fight Club,{u'url': u'https://images-na.ssl-images-amazon...,1403828,1999,True,feature,"{u'Gross:': 37023395, u'Opening Weekend:': 110...",63000000,37023395.0,11035485,[Drama]


In [398]:
#functions split up the Box Office Dictionaries into seperate columns for better parsing:
def get_budget(series):
    return(series['Budget:'])
def get_gross(series):
    return series['Gross:']
def get_opening_weekend(series):
    return series['Opening Weekend:']

# uses the above functions to set each box office metric to it's own column
df['Budget'] = df['Box Office'].apply(get_budget)
df['Gross'] = df['Box Office'].apply(get_gross)
df['Opening Weekend'] = df['Box Office'].apply(get_opening_weekend)
df.head()

,rating,tconst,title,image,num_votes,year,can_rate,type,Box Office,Budget,Gross,Opening Weekend
0,9.3,tt0111161,The Shawshank Redemption,{u'url': u'https://images-na.ssl-images-amazon...,1756854,1994,True,feature,"{u'Gross:': 28341469, u'Opening Weekend:': Non...",25000000,28341469.0,None
1,9.2,tt0068646,The Godfather,{u'url': u'https://images-na.ssl-images-amazon...,1199825,1972,True,feature,"{u'Gross:': 134821952, u'Opening Weekend:': 22...",6000000,134821952.0,223758
2,9.0,tt0071562,The Godfather: Part II,{u'url': u'https://images-na.ssl-images-amazon...,823982,1974,True,feature,"{u'Gross:': 57300000, u'Opening Weekend:': Non...",13000000,57300000.0,None
3,9.0,tt0468569,The Dark Knight,{u'url': u'https://images-na.ssl-images-amazon...,1739916,2008,True,feature,"{u'Gross:': 533316061, u'Opening Weekend:': 15...",185000000,533316061.0,158411483
4,8.9,tt0050083,12 Angry Men,{u'url': u'https://images-na.ssl-images-amazon...,471357,1957,True,feature,"{u'Gross:': None, u'Opening Weekend:': None, u...",350000,NaN,None


In [415]:
df['reviews text'] = df['tconst'].apply(get_reviews_text)

In [452]:
def value_is_int(series):
    return(type(series)==int)

In [453]:
df['budget_is_int'] = df['Budget'].apply(value_is_int)

In [460]:
df['year_as_int'] = df['year'].astype(int)

In [461]:
df['age'] = 2017-df['year_as_int']

In [462]:
valid_budget_df = df[df['budget_is_int']==True]
valid_budget_df

,rating,tconst,title,image,num_votes,year,can_rate,type,Box Office,Budget,Gross,Opening Weekend,Genres,reviews text,budget_is_int,year_as_int,age
0,9.3,tt0111161,The Shawshank Redemption,{u'url': u'https://images-na.ssl-images-amazon...,1756854,1994,True,feature,"{u'Gross:': 28341469, u'Opening Weekend:': Non...",25000000,28341469.0,None,"[Crime, Drama]",[<p>\nWhy do I want to write the 234th comment...,True,1994,23
1,9.2,tt0068646,The Godfather,{u'url': u'https://images-na.ssl-images-amazon...,1199825,1972,True,feature,"{u'Gross:': 134821952, u'Opening Weekend:': 22...",6000000,134821952.0,223758,"[Crime, Drama]",[<p><b>*** This review may contain spoilers **...,True,1972,45
2,9.0,tt0071562,The Godfather: Part II,{u'url': u'https://images-na.ssl-images-amazon...,823982,1974,True,feature,"{u'Gross:': 57300000, u'Opening Weekend:': Non...",13000000,57300000.0,None,"[Crime, Drama]",[<p>\n\nThis movie is way to be good to be lab...,True,1974,43
3,9.0,tt0468569,The Dark Knight,{u'url': u'https://images-na.ssl-images-amazon...,1739916,2008,True,feature,"{u'Gross:': 533316061, u'Opening Weekend:': 15...",185000000,533316061.0,158411483,"[Action, Crime, Drama, Thriller]",[<p>\nWe've been subjected to enormous amounts...,True,2008,9
4,8.9,tt0050083,12 Angry Men,{u'url': u'https://images-na.ssl-images-amazon...,471357,1957,True,feature,"{u'Gross:': None, u'Opening Weekend:': None, u...",350000,NaN,None,"[Crime, Drama]",[<p><b>*** This review may contain spoilers **...,True,1957,60
5,8.9,tt0108052,Schindler's List,{u'url': u'https://images-na.ssl-images-amazon...,900235,1993,True,feature,"{u'Gross:': 96067179, u'Opening Weekend:': u'£...",22000000,96067179.0,£1234591,"[Biography, Drama, History]","[<p>\nI know many of you, out there will disag...",True,1993,24
6,8.9,tt0110912,Pulp Fiction,{u'url': u'https://images-na.ssl-images-amazon...,1375727,1994,True,feature,"{u'Gross:': 107930000, u'Opening Weekend:': u'...",8000000,107930000.0,£4243233,"[Crime, Drama]","[<p>\nOne of the early scenes in ""Pulp Fiction...",True,1994,23
7,8.9,tt0167260,The Lord of the Rings: The Return of the King,{u'url': u'https://images-na.ssl-images-amazon...,1261755,2003,True,feature,"{u'Gross:': 377019252, u'Opening Weekend:': u'...",94000000,377019252.0,£15021761,"[Adventure, Drama, Fantasy]",[<p><b>*** This review may contain spoilers **...,True,2003,14
8,8.9,tt0060196,"The Good, the Bad and the Ugly",{u'url': u'https://images-na.ssl-images-amazon...,522499,1966,True,feature,"{u'Gross:': 6100000, u'Opening Weekend:': None...",1200000,6100000.0,None,[Western],[<p>\n\nA sprawling Western epic that follows ...,True,1966,51
9,8.8,tt0137523,Fight Club,{u'url': u'https://images-na.ssl-images-amazon...,1403828,1999,True,feature,"{u'Gross:': 37023395, u'Opening Weekend:': 110...",63000000,37023395.0,11035485,[Drama],[<p>\n\nWhen I first saw the previews for this...,True,1999,18


In [472]:
X = valid_budget_df[['num_votes','Budget','age']]
y = valid_budget_df['rating']
y = np.array(y).astype(int)

cv = StratifiedKFold(y, n_folds=3, shuffle=True, random_state=41)
rf = RandomForestClassifier()
rf.fit(X,y)

rf.feature_importances_

array([ 0.6757022 ,  0.12510534,  0.19919246])

In [473]:
cv = StratifiedKFold(y, n_folds=3, shuffle=True)

def score(model, name):
    s = cross_val_score(model, X, y, cv=cv, n_jobs=-1)
    print "{} Score:\t{:0.3} ± {:0.3}".format(name, s.mean().round(3), s.std().round(3))
    
score(rf, 'Random Forest')

Random Forest Score:	0.981 ± 0.007


# In Conclusion

For me this project turned out to be a two day exercise in data cleansing, web scraping and programming.  Recommendations for Netflix are to give me more time with the data and a team to review my work to spot errors and provide suggestions.

In [ ]:
# functions are below:

In [296]:
def get_soup(movie_id):
    base_url = 'http://www.imdb.com/title/'
    request = requests.get(base_url+movie_id)
    html = request.text
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [377]:
def get_box_office_info(movie_id):
    #print(movie_id)
    base_url = 'http://www.imdb.com/title/'
    request = requests.get(base_url+movie_id)
    html = request.text
    soup = BeautifulSoup(html, 'html.parser')
    
    Box_Office_Dict = {'Budget:': None, 'Gross:': None, 'Opening Weekend:': None}
    for i in soup.find_all('h4', {"class":'inline'}):
        if i.text in Box_Office_Dict.keys():
            x = i.next_element.next_element.strip()
            x = re.sub('\$','',x)
            x = re.sub(',','',x)
            x = re.sub(' ','',x)
            x = re.sub('[\(\[].*?[\)\]]','',x)
            x = re.sub('\n','',x)
            try:
                x = int(x)
            except:
                pass
            Box_Office_Dict[i.text] = x
    #Box_Office_Dict = clean_box_office(Box_Office_Dict)
    return(Box_Office_Dict)
#Box_Office_Dict

In [333]:
def get_genres(movie_id):
    genre_list = []
    base_url = 'http://www.imdb.com/title/'
    request = requests.get(base_url+movie_id)
    html = request.text
    soup = BeautifulSoup(html, 'html.parser')
    for i in soup.find_all('div',{'itemprop':'genre'}):
        j = i.find_all('a')
        for x in j:
            genre_list.append(x.next_element.strip())
    return (genre_list)

In [463]:
def get_reviews_text(movie_id):
    base_url = 'http://www.imdb.com/title/'
    request = requests.get(base_url+movie_id+'/reviews')
    html = request.text
    soup = BeautifulSoup(html, 'html.parser')
    paragraphs = soup.find_all('p', {'class':None})
    reviews = str(paragraphs[5:])
    reviews_string = str()
    for i in reviews:
        reviews_string += i
    
    return (reviews_string)
#get_reviews_text('tt0068646')

In [464]:
#my_text = get_reviews_text('tt0068646')
#my_text

In [447]:
#from nltk.stem import PorterStemmer
#stemmer = PorterStemmer()
#from sklearn.feature_extraction.text import CountVectorizer
#cvec = CountVectorizer(stop_words='english')
#my_text = get_reviews_text('tt0068646')
#cvec.fit(df['reviews text'])


#CVtest  = pd.DataFrame(cvec.transform(get_reviews_text('tt0068646')).todense(),columns=cvec.get_feature_names())

#CVtest.transpose().sort_values(0, ascending=False).head(10).transpose()

In [465]:
#professional_soup = get_reviews_text('tt0118799')
#paragraphs = professional_soup.find_all('p', {'class':None})
#reviews = paragraphs[5:]

In [267]:
def clean_box_office_string(box_office_string):
    if box_office_string != None:
        #print(box_office_string)
        box_office_string = re.sub('\$','',box_office_string)
        box_office_string = re.sub(' ','',box_office_string)
        box_office_string = re.sub(',','',box_office_string)
        box_office_string = re.sub('\n','',box_office_string)
        box_office_string = re.sub('\£','',box_office_string)
        box_office_string = re.sub('[\(\[].*?[\)\]]','',box_office_string)
        #box_office_string = int(box_office_string)
    return box_office_string
print df['Budget'][1]
#tt4572514['Opening Weekend:'] = re.sub('\$','',tt4572514['Opening Weekend:'])
#tt4572514['Opening Weekend:'] = re.sub(' ','',tt4572514['Opening Weekend:'])
#tt4572514['Opening Weekend:'] = re.sub('\n','',tt4572514['Opening Weekend:'])
#tt4572514['Opening Weekend:'] = re.sub("[\(\[].*?[\)\]]", "", tt4572514['Opening Weekend:'])

        $6,000,000        

      


In [95]:
def clean_box_office(box_office_dict):
    for i in box_office_dict:
        print(i, box_office_dict[i])
        #box_office_dict[i] = re.sub('\$','',box_office_dict[i])
        box_office_dict[i] = re.sub(' ','',box_office_dict[i])
        box_office_dict[i] = re.sub(',','',box_office_dict[i])
        box_office_dict[i] = re.sub('\n','',box_office_dict[i])
        box_office_dict[i] = re.sub('\£','',box_office_dict[i])
        box_office_dict[i] = re.sub('[\(\[].*?[\)\]]','',box_office_dict[i])
        box_office_dict[i] = int(box_office_dict[i])
    return box_office_dict

#tt4572514['Opening Weekend:'] = re.sub('\$','',tt4572514['Opening Weekend:'])
#tt4572514['Opening Weekend:'] = re.sub(' ','',tt4572514['Opening Weekend:'])
#tt4572514['Opening Weekend:'] = re.sub('\n','',tt4572514['Opening Weekend:'])
#tt4572514['Opening Weekend:'] = re.sub("[\(\[].*?[\)\]]", "", tt4572514['Opening Weekend:'])